In [2]:
!pip install crewai

  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached jsonref-1.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached pdfminer.six-20231228-py3-none-any.whl.metadata (4.2 kB)
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opentelemetry-instrumentation-fastapi to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 26.7 MB/s eta 0:00:00a 0:00:01
Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 43.7 MB/s eta 0:00:00
Using cached jsonref-1.1.0-py3-none-any.whl (9.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 36.7 MB/s eta 0:00:00
Using cached pdfminer.si

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from crewai import Agent, Task, Crew
import os

# Initialize FastAPI
app = FastAPI()

# Load environment variables (Set your API keys in env variables)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
CHROMA_PATH = "./chroma_db"

OPENAI_API_KEY = "sk-"

# Load embeddings and vector databases for each PDF agent
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

pdf1_db = Chroma(persist_directory=f"{CHROMA_PATH}/pdf1", embedding_function=embeddings)
# pdf2_db = Chroma(persist_directory=f"{CHROMA_PATH}/pdf2", embedding_function=embeddings)
# pdf3_db = Chroma(persist_directory=f"{CHROMA_PATH}/pdf3", embedding_function=embeddings)

# Define agents
llm = ChatOpenAI(
    model="mistralai/Pixtral-12B-2409",
    temperature=0,
    max_retries=2,
    api_key="...",
    base_url="http://0.0.0.0:9104/v1",
    # max_tokens = 2000
    # organization="...",
    # other params...
)
class Query(BaseModel):
    user_query: str

# PDF Agents
def create_pdf_agent(name, db):
    return Agent(
        role=name,
        goal=f"Answer queries based on {name}.",
        backstory=f"You are responsible for retrieving and answering questions from {name}.",
        llm=llm,
        memory=True,
        tools=[db]
    )

pdf_agents = {
    "Bot_A": create_pdf_agent("PDF_1", pdf1_db),
    # "Bot_B": create_pdf_agent("PDF 2", pdf2_db),
    # "Bot_C": create_pdf_agent("PDF 3", pdf3_db)
}

# Manager Agent (Router)
def route_query(query):
    relevant_agents = []
    for name, agent in pdf_agents.items():
        results = agent.tools[0].similarity_search(query, k=1)
        if results:
            relevant_agents.append(agent)
    return relevant_agents

# Response Aggregation Agent
def aggregate_responses(responses):
    return "\n".join(responses)

# FastAPI Endpoint
@app.post("/chat")
def chat(query: Query):
    agents = route_query(query.user_query)
    if not agents:
        return {"response": "No relevant information found."}
    
    # CrewAI Task execution
    tasks = [Task(description=query.user_query, agent=agent) for agent in agents]
    crew = Crew(agents=agents, tasks=tasks)
    responses = crew.kickoff()
    
    return {"response": aggregate_responses(responses)}

# Run with: uvicorn script_name:app --reload


# fastapi 
# uvicorn 
# langchain 
# langchain-community
# openai 
# chromadb 
# crewai
# starlette

/tmp/ipykernel_2962869/4020343674.py:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/home/vikasnr/miniconda3/envs/mispix_clone/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

/home/vikasnr/miniconda3/envs/mispix_clone/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/tmp/ipykernel_2962869/4020343674.py:21: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  pdf1_db = Chroma(persist_directory=f"{CHROMA_PATH}/pdf1", embedding_function=embeddings)
/tmp/ipykernel_2962869/4020343674.py:26: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated

ValidationError: 1 validation error for Agent
tools.0
  Input should be a valid dictionary or instance of BaseTool [type=model_type, input_value=<langchain_community.vect...bject at 0x7f85dc60ace0>, input_type=Chroma]
    For further information visit https://errors.pydantic.dev/2.9/v/model_type

In [4]:
import requests

def call_test_api():
    url = "http://localhost:9169"
    response = requests.get(url)
    
    if response.status_code == 200:
        print("API call successful")
        print("Response:", response.json())
    else:
        print("API call failed with status code:", response.status_code)

if __name__ == "__main__":
    call_test_api()

API call successful
Response: {'message': 'Hello, World!'}


In [4]:
!pip install PyPDF2

  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)


In [ ]:
import requests
import json
def query_llm_rest_pix(mlist,image=None):
        
        url = 'http://:9109/v1/chat/completions'
        headers = {
            'Content-Type': 'application/json',
            'Authorization': 'Bearer token'
        }
        data = {
            "model": "mistralai/Pixtral-12B-2409"
        }
        data.update({"messages": mlist})

        response = requests.post(url, headers=headers, data=json.dumps(data))
        # print(response.status_code)
        # print(response)
        if response.status_code == 400:
            return "I'm unsure about the answer. Please provide more context or ask a different question."
        else:
            return response.json()["choices"][0]['message']['content']


In [25]:
import os
from fastapi import FastAPI, Query
from pydantic import BaseModel
from typing import List
from crewai import Agent, Task, Crew, Process
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
# from crewai_tools import tool
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json

# llm = ChatOpenAI(
#     model="mistralai/Pixtral-12B-2409",
#     temperature=0,
#     max_retries=2,
#     api_key="...",
#     base_url="http://0.0.0.0:9109/v1",
#     # max_tokens = 2000
#     # organization="...",
#     # other params...
# )

from chromadb import HttpClient
client = HttpClient(host="localhost", port=9158)

class VectorDBHandler:
    def __init__(self, collection_name: str):
        self.client = HttpClient(host="localhost", port=9158)
        self.collection = self.client.get_or_create_collection(name=collection_name)
        self.encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

    def add_texts(self, texts: List[str], metadata: List[dict] = None):
        embeddings = self.encoder.encode(texts).tolist()
        ids = [str(i) for i in range(len(texts))]
        print(metadata)
        self.collection.add(
            documents=texts,
            embeddings=embeddings,
            # metadatas=metadata if metadata else [{}] * len(texts),
            ids=ids
        )
from sentence_transformers import SentenceTransformer

# embeddings = SentenceTransformer("all-MiniLM-L6-v2")
encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

CHROMA_PATH = '/home/vikasnr/codebase/crsl/crew_gpt/chroma_db'
# Initialize Vector Stores for each PDF
# vector_store_1 = Chroma(persist_directory=f"{CHROMA_PATH}/db_pdf1", embedding_function=embeddings)
# vector_store_2 = Chroma(persist_directory=f"{CHROMA_PATH}/db_pdf2", embedding_function=embeddings)
# vector_store_3 = Chroma(persist_directory=f"{CHROMA_PATH}db_pdf3", embedding_function=embeddings)
# pdf1_db = Chroma(persist_directory=f"{CHROMA_PATH}/pdf1", embedding_function=embeddings)
# pdf2_db = Chroma(persist_directory=f"{CHROMA_PATH}/pdf2", embedding_function=embeddings)
# pdf3_db = Chroma(persist_directory=f"{CHROMA_PATH}/pdf3", embedding_function=embeddings)

from PyPDF2 import PdfReader
from typing import List
from langchain.text_splitter import RecursiveCharacterTextSplitter

def extract_text_from_pdf(pdf_path: str) -> List[str]:
    reader = PdfReader(pdf_path)
    chunks = []
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

    for page in reader.pages:
        text = page.extract_text()
        paragraphs = text.split('\n\n')
        for paragraph in paragraphs:
            if paragraph.strip():
                chunks.extend(splitter.split_text(paragraph.strip()))
    print("chunks--------------------------------------\n",len(chunks))
    return chunks

# Function to load, chunk, and store PDFs into vector stores
def process_and_store_pdf(pdf_path: str, collection_name: str):
    # Load PDF
    chunks = extract_text_from_pdf(pdf_path)
    embeddings = encoder.encode(chunks).tolist()
    ids = [str(i) for i in range(len(chunks))]
    collection = client.get_or_create_collection(name=collection_name)
    collection.add(
            documents=chunks,
            embeddings=embeddings,
            # metadatas=metadata if metadata else [{}] * len(texts),
            ids=ids
        )

    # Store chunks in vector store


    # Extract keywords from chunks using LLM
    combined_chunks = " ".join(chunks)

    # print(chunk)
    prompt = f"""Extract only the keywords from the following text. Return response in a python list. Do not include any other text in the response:\n\n
            {combined_chunks}\n\n
            example response:
            ['keyword1', 'keyword2', 'keyword3']
            """
    # print(prompt)
    
    response = query_llm_rest_pix([{"role": "user", "content": [{"type": "text", "text": prompt}]}])
    print(response)
    import re
    import ast   
    match = re.search(r"\[.*\]", response, re.DOTALL)

    if match:
        keyword_list = ast.literal_eval(match.group(0))  # Safely convert to Python list
        print(keyword_list)
    else:
        keyword_list = []
        print("No list found in response.")
    # Save keywords to a dictionary
    keywords_dict = {}
    file_name = os.path.basename(pdf_path)
    keywords_dict[file_name] = keyword_list

    # Open the file and update the dictionary
    keywords_file = "/home/vikasnr/codebase/crsl/crew_gpt/keywords.json"
    if os.path.exists(keywords_file):
        with open(keywords_file, "r") as f:
            existing_keywords = json.load(f)
    else:
        existing_keywords = {}

    existing_keywords.update(keywords_dict)

    with open(keywords_file, "w") as f:
        json.dump(existing_keywords, f, indent=4)

# Process and store each PDF
process_and_store_pdf("/home/vikasnr/codebase/crsl/pdfs/crazyones-pdfa.pdf", "crazyones")
process_and_store_pdf("/home/vikasnr/codebase/crsl/pdfs/midterm_report.pdf", "midterm")
process_and_store_pdf("/home/vikasnr/codebase/crsl/pdfs/spores_manual.pdf", "spores")

chunks--------------------------------------
 1
['Crazy', 'Ones', 'October', '1998', 'Heres', 'to', 'the', 'crazy', 'ones', 'misfits', 'rebels', 'troublemakers', 'round', 'peg', 'square', 'holes', 'see', 'things', 'differently', 'rules', 'respect', 'status', 'quo', 'quote', 'disagree', 'glorify', 'vilify', 'ignore', 'change', 'things', 'invent', 'imagine', 'heal', 'explore', 'create', 'inspire', 'human', 'race', 'forward', 'crazy', 'empty', 'canvas', 'work', 'art', 'silence', 'song', 'red', 'planet', 'laboratory', 'wheels', 'genius', 'think', 'change', 'world', 'crazy', 'enough', 'see', 'never', 'written', 'gaz', 'red', 'planet', 'laboratory', 'wheels', 'tools', 'kind', 'people']
['Crazy', 'Ones', 'October', '1998', 'Heres', 'to', 'the', 'crazy', 'ones', 'misfits', 'rebels', 'troublemakers', 'round', 'peg', 'square', 'holes', 'see', 'things', 'differently', 'rules', 'respect', 'status', 'quo', 'quote', 'disagree', 'glorify', 'vilify', 'ignore', 'change', 'things', 'invent', 'imagine', 

In [3]:
# !pip install crewai
!pip install crewai-tools

  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached pytube-15.0.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached gptcache-0.1.44-py3-none-any.whl.metadata (24 kB)
  Using cached langchain_openai-0.2.14-py3-none-any.whl.metadata (2.7 kB)
  Using cached pysbd-0.3.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached schema-0.7.7-py2.py3-none-any.whl.metadata (34 kB)
  Using cached deprecation-2.1.0-py2.py3-none-any.whl.metadata (4.6 kB)
  Using cached nodeenv-1.9.1-py2.py3-none-any.whl.metadata (21 kB)
  Using cached portalocker-2.10.1-py3-none-any.whl.metadata (8.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.5/548.5 kB 17.2 MB/s eta 0:00:00
Using cached docker-7.1.0-py3-none-any.whl (147 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.2/33.2 MB 9.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 50.9 MB/s eta 0:00:00
Using cached pytube

In [ ]:
from langchain_community.chat_models import ChatOpenAI
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

import os
from fastapi import FastAPI, Query
from pydantic import BaseModel
from typing import List
from crewai import Agent, Task, Crew, Process
# from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from chromadb import HttpClient
from sentence_transformers import SentenceTransformer
# Initialize FastAPI
# app = FastAPI()

# Initialize LLM
llm = ChatOpenAI(
    model="mistralai/Mistral-Small-24B-Instruct-2501",

    temperature=0,
    max_retries=2,
    api_key="...",
    base_url="http://0.0.0.0:9150/v1",
)

llm.invoke("Hello how are you?")


NotFoundError: Error code: 404 - {'object': 'error', 'message': 'The model `mistralai/Mistral-Small-24B-Instruct-251` does not exist.', 'type': 'NotFoundError', 'param': None, 'code': 404}

cf.py

In [2]:
import os
from fastapi import FastAPI, Query,HTTPException
from pydantic import BaseModel
from typing import List
from crewai import Agent, Task, Crew, Process,LLM
# from langchain.vectorstores import Chroma
from crewai.tools import tool
import json
# from langchain_core.tools import tool

# from langchain.chat_models import ChatOpenAI
from langchain_community.chat_models import ChatOpenAI
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

import os
from fastapi import FastAPI, Query
from pydantic import BaseModel
from typing import List
from crewai import Agent, Task, Crew, Process
# from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from chromadb import HttpClient
from sentence_transformers import SentenceTransformer
# Initialize FastAPI
# app = FastAPI()

# Initialize LLM
# llm = ChatOpenAI(
#     model="mistralai/Pixtral-12B-2409",
#     temperature=0,
#     max_retries=2,
#     api_key="...",
#     base_url="http://0.0.0.0:9109/v1",
# )



llm = LLM(
    model="mistral/Mistral-Small-24B-Instruct-2501",
    base_url="http://0.0.0.0:9150/v1",
    api_key="your-api-key"
)

# Initialize Vector Database Client
client = HttpClient(host="localhost", port=9158)
encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Define Tools
@tool('query_vector_store')
def query_vector_store(query: str, collection_name: str) -> str:
    """Search a vector store for answers."""
    collection = client.get_collection(name=collection_name)
    docs = collection.query(query, n_results=3)
    return " ".join([doc.page_content for doc in docs])

@tool('route_query')
def route_query(query: str) -> List[str]:
    """Determine which document agents should handle the query."""
    keywords_file = "/home/vikasnr/codebase/crsl/crew_gpt/keywords.json"
    if os.path.exists(keywords_file):
        with open(keywords_file, "r") as f:
            keywords_data = json.load(f)
    else:
        keywords_data = {}
    
    selected_agents = []
    for file_name, keywords in keywords_data.items():
        if any(word in query.lower() for word in keywords):
            selected_agents.append(file_name.replace(".pdf", "_Agent"))
    
    return selected_agents if selected_agents else ["crazyones_Agent", "midterm_Agent", "spores_Agent"]

@tool
def aggregate_answers(answers: List[str]) -> str:
    """Combine multiple answers into a final response."""
    return " ".join(answers)

# Define Agents
manager_agent = Agent(
    name="Manager_Agent",
    role="Query Router",
    goal="Decide which agents should answer the question.",
    backstory="I am an AI assistant that helps route questions to the appropriate document agents.",
    tools=[route_query],
    llm=llm,
    allow_delegation=True,
)

doc_agents = {
    "crazyones_Agent": Agent(
    name="PDF_1_Agent",
    role="Document Expert",
    goal="Answer questions using PDF 1 data.",
    backstory="An AI assistant trained on PDF 1's content to extract and provide relevant answers.",

    tools=[query_vector_store],
    llm=llm,
),
    "midterm_Agent": Agent(
    name="PDF_2_Agent",
    role="Document Expert",
    goal="Answer questions using PDF 2 data.",
    backstory="An AI assistant trained on PDF 2's content to extract and provide relevant answers.",

    tools=[query_vector_store],
    llm=llm,
),
    "spores_Agent": Agent(
    name="PDF_3_Agent",
    role="Document Expert",
    goal="Answer questions using PDF 3 data.",
    backstory="An AI assistant trained on PDF 3's content to extract and provide relevant answers.",

    tools=[query_vector_store],
    llm=llm,
)

}

aggregator_agent = Agent(
    name="Aggregator_Agent",
    role="Answer Aggregator",
    goal="Combine answers from document agents and provide a final response.",
    backstory="An AI assistant that aggregates answers from multiple document agents to provide a comprehensive response.",
    tools=[aggregate_answers],
    llm=llm,
)

task_1 = Task(description="Retrieve relevant info from PDF 1", expected_output="Extracted text.", agent=doc_agents["crazyones_Agent"])
task_2 = Task(description="Retrieve relevant info from PDF 2", expected_output="Extracted text.", agent=doc_agents["midterm_Agent"])
task_3 = Task(description="Retrieve relevant info from PDF 3", expected_output="Extracted text.", agent=doc_agents["spores_Agent"])
manager_task = Task(description="Determine which agents should be queried.", expected_output="List of agents.", agent=manager_agent)
aggregator_task = Task(description="Aggregate the answers.", expected_output="Final response.", agent=aggregator_agent)



# Define Crew
crew = Crew(
    agents=[manager_agent,*doc_agents.values(), aggregator_agent],
    tasks=[manager_task,task_1, task_2, task_3, aggregator_task],
    process=Process.sequential,
    manager_llm=llm,
    # manager_agent=manager_agent
)

# Define API Request Model
class QueryRequest(BaseModel):
    question: str


ModuleNotFoundError: No module named 'starlette'

In [ ]:
# llm = LLM(
#     model="mistral/Mistral-Small-24B-Instruct-2501",
#     base_url="http://0.0.0.0:9150/v1",
#     api_key = "your-api"
# )
# llm.call("Hello how are you?")
from crewai import LLM
# llm = LLM(
#     model="mistral/mistral-large-latest",
#     temperature=0.7,
#    
# llm.call("Hello how are you?")



 
[2025-03-14 18:21:19][🤖 LLM CALL STARTED]: 2025-03-14 18:21:19.335191
 
[2025-03-14 18:21:20][✅ LLM CALL COMPLETED]: 2025-03-14 18:21:20.398989


"Hello! I'm functioning as intended, thank you. How about you? How's your day going?"

In [54]:

question = "Who are the crazy ones?"
crew.kickoff()
# relevant_agents = crew.query(manager_agent, route_query, question)

# answers = [
#     crew.query(doc_agents[agent], query_vector_store, question, collection_name=agent.replace("_Agent", ""))
#     for agent in relevant_agents
# ]

# final_response = crew.query(aggregator_agent, aggregate_answers, answers)


ERROR:root:LiteLLM call failed: litellm.NotFoundError: NotFoundError: MistralException - Error code: 404 - {'object': 'error', 'message': 'The model `Mistral-Small-24B-Instruct-2501` does not exist.', 'type': 'NotFoundError', 'param': None, 'code': 404}


 
[2025-03-14 13:35:50][🚀 CREW 'CREW' STARTED, B89D17DD-F615-4991-8DDA-285DF3F1DF34]: 2025-03-14 13:35:50.599037
 
[2025-03-14 13:35:50][📋 TASK STARTED: DETERMINE WHICH AGENTS SHOULD BE QUERIED.]: 2025-03-14 13:35:50.612004
 
[2025-03-14 13:35:50][🤖 AGENT 'QUERY ROUTER' STARTED TASK]: 2025-03-14 13:35:50.613378
 
[2025-03-14 13:35:50][🤖 LLM CALL STARTED]: 2025-03-14 13:35:50.613585


LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 
[2025-03-14 13:35:50][❌ LLM CALL FAILED: 'LITELLM.NOTFOUNDERROR: NOTFOUNDERROR: MISTRALEXCEPTION - ERROR CODE: 404 - {'OBJECT': 'ERROR', 'MESSAGE': 'THE MODEL `MISTRAL-SMALL-24B-INSTRUCT-2501` DOES NOT EXIST.', 'TYPE': 'NOTFOUNDERROR', 'PARAM': NONE, 'CODE': 404}']: 2025-03-14 13:35:50.669513
 Error during LLM call: litellm.NotFoundError: NotFoundError: MistralException - Error code: 404 - {'object': 'error', 'message': 'The model `Mistral-Small-24B-Instruct-2501` does n

NotFoundError: litellm.NotFoundError: NotFoundError: MistralException - Error code: 404 - {'object': 'error', 'message': 'The model `Mistral-Small-24B-Instruct-2501` does not exist.', 'type': 'NotFoundError', 'param': None, 'code': 404}

In [41]:
!pip list | grep llmlite

In [ ]:
# import base64
import json
import boto3

# with open(
#     r"C:\Users\BGH52648\work\modules\LLM\IA\reader\FAA-2017-0477-0004_attachment_1\page_15.png",
#     "rb",
# ) as image_file:
#     image_bytes = image_file.read()

bedrock_client = boto3.client(
    "bedrock-runtime",
    region_name="us-east-1",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)

# encoded_image = base64.b64encode(image_bytes).decode("utf-8")
prompt = "Extract table in the image and create a json object"
body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    # {
                    #     "type": "image",
                    #     "source": {
                    #         "type": "base64",
                    #         "media_type": "image/jpeg",
                    #         # "data": encoded_image,
                    #     },
                    # },
                    {"type": "text", "text": "What is economics?"},
                ],
            }
        ],
    }
)

response = bedrock_client.invoke_model(
    modelId=MODEL, body=body
)

response_body = json.loads(response.get("body").read())

# print(response_body)
import pprint as pp

pp.pprint(response_body["content"][0]["text"])

ClientError: An error occurred (UnrecognizedClientException) when calling the InvokeModel operation: The security token included in the request is invalid.

In [ ]:
# import base64
import json
import boto3



bedrock_client = boto3.client(
    "bedrock-runtime",
    region_name="us-east-1",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
)

# encoded_image = base64.b64encode(image_bytes).decode("utf-8")
prompt = "Extract table in the image and create a json object"
body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    # {
                    #     "type": "image",
                    #     "source": {
                    #         "type": "base64",
                    #         "media_type": "image/jpeg",
                    #         # "data": encoded_image,
                    #     },
                    # },
                    {"type": "text", "text": "What is economics?"},
                ],
            }
        ],
    }
)

response = bedrock_client.invoke_model(
    modelId=MODEL, body=body
)

response_body = json.loads(response.get("body").read())

# print(response_body)
import pprint as pp

pp.pprint(response_body["content"][0]["text"])

('Economics is a social science that studies how individuals, businesses, '
 'governments, and societies allocate scarce resources to satisfy their '
 'unlimited wants and needs. It focuses on the production, distribution, and '
 'consumption of goods and services, as well as the factors that influence '
 'these processes.\n'
 '\n'
 'Key aspects of economics include:\n'
 '\n'
 '1. Microeconomics: The study of individual economic units, such as '
 'households and firms, and how they make decisions.\n'
 '\n'
 '2. Macroeconomics: The study of the overall economy, including issues like '
 'inflation, unemployment, and economic growth.\n'
 '\n'
 '3. Supply and demand: The fundamental principles that govern market behavior '
 'and price determination.\n'
 '\n'
 '4. Market structures: Different types of markets, such as perfect '
 'competition, monopoly, and oligopoly.\n'
 '\n'
 '5. Economic policy: Government interventions to influence economic outcomes, '
 'such as fiscal and monetary polic

In [ ]:
import os
from litellm import completion


response = completion(
  model="bedrock/anthropic.claude-3-sonnet-20240229-v1:0",
  messages=[{ "content": "Hello, how are you?","role": "user"}]
)
response

ModelResponse(id='chatcmpl-8b71f9e7-7721-4945-8307-a2becafc6977', created=1741960854, model='anthropic.claude-3-sonnet-20240229-v1:0', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content="Hello! As an AI language model, I don't have subjective experiences like feelings, but I'm operating properly and ready to assist you with any questions or tasks you may have. How can I help you today?", role='assistant', tool_calls=None, function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=47, prompt_tokens=13, total_tokens=60, completion_tokens_details=None, prompt_tokens_details=None))

In [ ]:
from crewai import LLM


llm = LLM(
    model="bedrock/anthropic.claude-3-sonnet-20240229-v1:0",
)
llm.call("Hello how are you?")

 
[2025-03-14 19:34:06][🤖 LLM CALL STARTED]: 2025-03-14 19:34:06.104359
 
[2025-03-14 19:34:08][✅ LLM CALL COMPLETED]: 2025-03-14 19:34:08.322409


"Hello! As an AI language model, I don't have feelings, but I'm operating properly and ready to assist you with any questions or tasks you may have. How can I help you today?"